<a href="https://colab.research.google.com/github/kasteway/Stock_Prediction/blob/main/StockMarket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 32.4 MB/s 
     |████████████████████████████████| 62 kB 273 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf

In [3]:
data = yf.download("GOOGL" , start = "2018-01-01" , interval = '1d')

[*********************100%***********************]  1 of 1 completed


In [4]:
data.shape

(1142, 6)

In [5]:
data.head(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,1053.020020,1075.979980,1053.020020,1073.209961,1073.209961,1588300
2018-01-03,1073.930054,1096.099976,1073.430054,1091.520020,1091.520020,1565900
2018-01-04,1097.089966,1104.079956,1094.260010,1095.760010,1095.760010,1302600


Understanding Trends with in the Data

In [6]:
# Sort the data points based on indexes just for confirmation 
data.sort_index(inplace = True)

In [7]:
# Remove any duplicate index 
data = data.loc[~data.index.duplicated(keep='first')]

In [8]:
data.tail(3)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-07-13,2234.020020,2283.919922,2224.030029,2227.070068,2227.070068,2186100
2022-07-14,2204.209961,2221.860107,2167.459961,2207.350098,2207.350098,1847500
2022-07-15,2240.010010,2262.810059,2218.000000,2230.125000,2230.125000,981059


In [9]:
# Check for missing values 
data.isnull().sum()

Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [10]:
# Get the statistics of the data
data.describe()

,Open,High,Low,Close,Adj Close,Volume
count,1142.000000,1142.000000,1142.000000,1142.000000,1142.000000,1.142000e+03
mean,1680.278791,1698.385741,1661.913453,1680.619032,1680.619032,1.788678e+06
std,633.824160,639.559235,626.564106,632.913542,632.913542,8.176421e+05
min,984.320007,1012.119995,977.659973,984.669983,984.669983,4.656000e+05
25%,1159.420013,1175.962494,1150.659973,1164.422485,1164.422485,1.265100e+06
50%,1401.570007,1420.500000,1387.309998,1405.895020,1405.895020,1.584500e+06
75%,2263.165039,2288.134888,2240.242554,2268.860046,2268.860046,2.046650e+06
max,3025.000000,3030.929932,2977.979980,2996.770020,2996.770020,6.658900e+06


In [11]:
import plotly.graph_objects as go

# Check the trend in Closing Values 
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Close'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900, 
                  xaxis_title='Date' , yaxis_title='Close')
fig.show()

In [12]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Volume'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900, 
                  xaxis_title='Date' , yaxis_title='Volume')
fig.show()

Data Preparation

In [13]:
from sklearn.preprocessing import MinMaxScaler 
import pickle 
from tqdm.notebook import tnrange

In [14]:
# Filter only required data 
data = data[['Close' , 'Volume']]
data.head(3)

,Close,Volume
Date,,
2018-01-02,1073.209961,1588300
2018-01-03,1091.520020,1565900
2018-01-04,1095.760010,1302600


In [15]:
# Confirm the Testing Set length 
test_length = data[(data.index >= '2020-09-01')].shape[0]

In [16]:
def CreateFeatures_and_Targets(data, feature_length):
    X = []
    Y = []

    for i in tnrange(len(data) - feature_length): 
        X.append(data.iloc[i : i + feature_length,:].values)
        Y.append(data["Close"].values[i+feature_length])

    X = np.array(X)
    Y = np.array(Y)

    return X , Y

In [17]:
X , Y = CreateFeatures_and_Targets(data , 32)

  0%|          | 0/1110 [00:00<?, ?it/s]

In [18]:
# Check the shapes
X.shape , Y.shape

((1110, 32, 2), (1110,))

In [19]:
Xtrain , Xtest , Ytrain , Ytest = X[:-test_length] , X[-test_length:] , Y[:-test_length] , Y[-test_length:]

In [20]:
# Check Training Dataset Shape 
Xtrain.shape , Ytrain.shape

((639, 32, 2), (639,))

In [21]:
# Check Testing Dataset Shape
Xtest.shape , Ytest.shape

((471, 32, 2), (471,))

In [22]:
# Create a Scaler to Scale Vectors with Multiple Dimensions 
class MultiDimensionScaler():
    def __init__(self):
        self.scalers = []

    def fit_transform(self , X):
        total_dims = X.shape[2]
        for i in range(total_dims):
            Scaler = MinMaxScaler()
            X[:, :, i] = Scaler.fit_transform(X[:, :, i])
            self.scalers.append(Scaler)
        return X

    def transform(self , X):
        for i in range(X.shape[2]):
            X[:, :, i] = self.scalers[i].transform(X[:,:,i])
        return X 

In [23]:
Feature_Scaler = MultiDimensionScaler()
Xtrain = Feature_Scaler.fit_transform(Xtrain)
Xtest = Feature_Scaler.transform(Xtest)

In [24]:
Target_Scaler = MinMaxScaler()
Ytrain = Target_Scaler.fit_transform(Ytrain.reshape(-1,1))
Ytest = Target_Scaler.transform(Ytest.reshape(-1,1))

In [25]:
def save_object(obj , name : str):
    pickle_out = open(f"{name}.pck","wb")
    pickle.dump(obj, pickle_out)
    pickle_out.close()

def load_object(name : str):
    pickle_in = open(f"{name}.pck","rb")
    data = pickle.load(pickle_in)
    return data

In [26]:
# Save your objects for future purposes 
save_object(Feature_Scaler , "Feature_Scaler")
save_object(Target_Scaler , "Target_Scaler")

Model Building

In [27]:
from tensorflow.keras.callbacks import ModelCheckpoint , ReduceLROnPlateau

save_best = ModelCheckpoint("best_weights.h5", monitor='val_loss', save_best_only=True, save_weights_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25,patience=4, min_lr=0.00001,verbose = 1)

In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout , LSTM , Bidirectional

model = Sequential()

model.add(Bidirectional(LSTM(512 ,return_sequences=True , recurrent_dropout=0.1, input_shape=(32, 2))))
model.add(LSTM(256 ,recurrent_dropout=0.1))
model.add(Dropout(0.3))
model.add(Dense(64 , activation='elu'))
model.add(Dropout(0.3))
model.add(Dense(32 , activation='elu'))
model.add(Dense(1 , activation='linear'))

In [29]:
#optimizer = tf.keras.optimizers.Adam(learning_rate=0.002)
optimizer = tf.keras.optimizers.SGD(learning_rate = 0.002)
model.compile(loss='mse', optimizer=optimizer)

In [30]:
history = model.fit(Xtrain, Ytrain,
            epochs=10,
            batch_size = 1,
            verbose=1,
            shuffle=False ,
            validation_data=(Xtest , Ytest),
            callbacks=[reduce_lr , save_best])

Epoch 1/10
639/639 [==============================] - 800s 1s/step - loss: 0.0076 - val_loss: 0.9368 - lr: 0.0020
Epoch 2/10
639/639 [==============================] - 767s 1s/step - loss: 0.0086 - val_loss: 0.8964 - lr: 0.0020
Epoch 3/10
639/639 [==============================] - 765s 1s/step - loss: 0.0070 - val_loss: 0.6256 - lr: 0.0020
Epoch 4/10
639/639 [==============================] - 765s 1s/step - loss: 0.0063 - val_loss: 0.5909 - lr: 0.0020
Epoch 5/10
639/639 [==============================] - 767s 1s/step - loss: 0.0056 - val_loss: 0.4526 - lr: 0.0020
Epoch 6/10
639/639 [==============================] - 766s 1s/step - loss: 0.0054 - val_loss: 0.4739 - lr: 0.0020
Epoch 7/10
639/639 [==============================] - 777s 1s/step - loss: 0.0051 - val_loss: 0.4767 - lr: 0.0020
Epoch 8/10
639/639 [==============================] - 789s 1s/step - loss: 0.0048 - val_loss: 0.3830 - lr: 0.0020
Epoch 9/10
639/639 [==============================] - 788s 1s/step - loss: 0.0047 - val_

In [31]:
# Checking the model Structure 
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (1, 32, 1024)            2109440   
 l)                                                              
                                                                 
 lstm_1 (LSTM)               (1, 256)                  1311744   
                                                                 
 dropout (Dropout)           (1, 256)                  0         
                                                                 
 dense (Dense)               (1, 64)                   16448     
                                                                 
 dropout_1 (Dropout)         (1, 64)                   0         
                                                                 
 dense_1 (Dense)             (1, 32)                   2080      
                                                        

In [32]:
# Load the best weights
model.load_weights("best_weights.h5")

Visualize prediction on Test Set

In [33]:
Predictions = model.predict(Xtest)

In [34]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Ytest)

In [35]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [36]:
# Creating Sample Test Dataframe
test_dataframe_dict = {'Actual' : list(Actual) , 'Predicted' : list(Predictions)}
test_df = pd.DataFrame.from_dict(test_dataframe_dict)

test_df.index = data.index[-test_length:]

In [37]:
test_df.head()

,Actual,Predicted
Date,,
2020-09-01,1655.079956,1629.954346
2020-09-02,1717.390015,1636.509399
2020-09-03,1629.510010,1642.794434
2020-09-04,1581.209961,1641.997314
2020-09-08,1523.599976,1638.252563


In [38]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = test_df.index , y = Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x = test_df.index , y = Predictions , mode = 'lines' , name='Predicted'))
fig.show()

Visualize Prediction on whole data

In [39]:
Total_features = np.concatenate((Xtrain , Xtest) , axis = 0)

In [40]:
Total_Targets = np.concatenate((Ytrain , Ytest) , axis = 0)

In [41]:
Predictions = model.predict(Total_features)

In [42]:
Predictions = Target_Scaler.inverse_transform(Predictions)
Actual = Target_Scaler.inverse_transform(Total_Targets)

In [43]:
Predictions = np.squeeze(Predictions , axis = 1)
Actual = np.squeeze(Actual , axis = 1)

In [44]:
# Check the trend in Volume Traded
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = Actual , mode = 'lines' , name='Actual'))
fig.add_trace(go.Scatter(x = data.index , y = Predictions , mode = 'lines' , name='Predicted'))
fig.show()